In [3]:
import torch
from torchvision import datasets, transforms
from torchvision.transforms import v2
from torch.utils.data import DataLoader, Dataset, Subset, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as  np
from torchvision import utils
import matplotlib.pyplot as plt
import os

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [5]:
!unzip "Train_dataset_small.zip" -d "dataset"

Archive:  Train_dataset_small.zip
   creating: dataset/Train_dataset_new/
   creating: dataset/Train_dataset_new/airport_terminal/
  inflating: dataset/Train_dataset_new/airport_terminal/00000001.jpg  
  inflating: dataset/Train_dataset_new/airport_terminal/00000002.jpg  
  inflating: dataset/Train_dataset_new/airport_terminal/00000003.jpg  
  inflating: dataset/Train_dataset_new/airport_terminal/00000004.jpg  
  inflating: dataset/Train_dataset_new/airport_terminal/00000005.jpg  
  inflating: dataset/Train_dataset_new/airport_terminal/00000006.jpg  
  inflating: dataset/Train_dataset_new/airport_terminal/00000007.jpg  
  inflating: dataset/Train_dataset_new/airport_terminal/00000008.jpg  
  inflating: dataset/Train_dataset_new/airport_terminal/00000009.jpg  
  inflating: dataset/Train_dataset_new/airport_terminal/00000010.jpg  
  inflating: dataset/Train_dataset_new/airport_terminal/00000011.jpg  
  inflating: dataset/Train_dataset_new/airport_terminal/00000012.jpg  
  inflating: data

In [21]:
path='dataset/Train_dataset_new'
n_channels = 3
n_classes = 10
batch_size = 64
learning_rate = 0.001
transform = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

In [22]:
dataset = datasets.ImageFolder(root=path, transform=transform)

# Split the dataset into train, test and validation.
total_dataset_len = len(dataset)
train_dataset_len = int(0.7 * total_dataset_len)
val_dataset_len = int(0.15 * total_dataset_len)
test_dataset_len = total_dataset_len - train_dataset_len - val_dataset_len
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_dataset_len, val_dataset_len, test_dataset_len])

train_loader = DataLoader(dataset=train_dataset, batch_size=64, num_workers=2, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=64, num_workers=2, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, num_workers=2, shuffle=False)
print(len(train_loader))

532


In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

class AlaxNet_custom(nn.Module):
    def __init__(self, in_channels, out_classes):
        super(AlaxNet_custom, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=64, kernel_size=11, stride=4, padding=2)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=192, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm2d(192)
        self.conv3 = nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(384)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        
        self.relu = nn.ReLU(inplace=True)
        self.maxPool = nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(in_features=9216, out_features=4096)
        self.fc2 = nn.Linear(in_features=4096, out_features=4096)
        self.fc3 = nn.Linear(in_features=4096, out_features=out_classes)

    def forward(self, x):
        img = self.bn1(self.conv1(x))
        img = self.relu(img)
        img = self.maxPool(img)
        # 2nd
        img = self.bn2(self.conv2(img))
        img = self.relu(img)
        img = self.maxPool(img)
        # 3rd
        img = self.bn3(self.conv3(img))
        img = self.relu(img)
        # 4th
        img = self.bn4(self.conv4(img))
        img = self.relu(img)
        # 5th
        img = self.bn5(self.conv5(img))
        img = self.relu(img)
        img = self.maxPool(img)
        img = torch.flatten(img, 1)
        # 6th
        img = self.dropout(img)
        img = self.fc1(img)
        img = self.relu(img)
        # 7th
        img = self.dropout(img)
        img = self.fc2(img)
        img = self.relu(img)
        # output
        img = self.fc3(img)
        return img



# Create an instance of the model
model = AlaxNet_custom(3, 10)
model.to(device)

# Print the model architecture
print(model)

# Reference: https://blog.paperspace.com/alexnet-pytorch/
# Reference: https://github.com/dansuh17/alexnet-pytorch/blob/d0c1b1c52296ffcbecfbf5b17e1d1685b4ca6744/model.py#L40

AlaxNet_custom(
  (conv1): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxPool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.

In [32]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, n_epochs):
    train_losses, train_accs = [], []
    val_losses, val_accs = [], []

    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_loss = train_loss / len(train_loader)
        train_acc = (train_correct / train_total) * 100
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        if scheduler:
            scheduler.step()

        # Validation
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(val_loader)
        val_acc = (val_correct / val_total) * 100
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        print(f'Epoch {epoch+1}/{n_epochs}:')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

    return train_losses, train_accs, val_losses, val_accs

In [33]:
criterion = nn.CrossEntropyLoss().cuda()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [ ]:
train_losses, train_accs, val_losses, val_accs = train_model(model, train_loader, val_loader, criterion, optimizer,scheduler, 10)

Epoch 1/10:
Train Loss: 2.4265, Train Acc: 15.4291
Val Loss: 2.1850, Val Acc: 17.4849
Epoch 2/10:
Train Loss: 2.1841, Train Acc: 17.7328
Val Loss: 2.1588, Val Acc: 19.8300
Epoch 3/10:
Train Loss: 2.1660, Train Acc: 19.4223
Val Loss: 2.1270, Val Acc: 21.7087
Epoch 4/10:
Train Loss: 2.1308, Train Acc: 20.8445
Val Loss: 2.1502, Val Acc: 21.5304
Epoch 5/10:
Train Loss: 2.1074, Train Acc: 22.4018
Val Loss: 2.2133, Val Acc: 20.2551
Epoch 6/10:
Train Loss: 2.0704, Train Acc: 24.1413
Val Loss: 2.0183, Val Acc: 27.8387
Epoch 7/10:
Train Loss: 2.0138, Train Acc: 26.2598
Val Loss: 1.9964, Val Acc: 27.8250
